In [1]:
import sys
import os

import sys
import os

# Get the absolute path to the `src` directory
src_path = os.path.abspath(os.path.join('..', 'src'))

# Add the `src` directory to the Python path
if src_path not in sys.path:
    sys.path.append(src_path)

import pandas as pd
import numpy as np
from joblib import load
from data_processing import fetch_stock_data, calculate_technical_indicators

# Function to predict tomorrow's stock movement
def predict_tomorrow_xgboost(ticker, feature_subset=None):
    """
    Predict tomorrow's stock movement for a given ticker using a pre-trained XGBoost model.
    
    Args:
    ticker: str, The stock ticker symbol.
    feature_subset: list of str, Features used during model training.
    
    Returns:
    prediction: str, 'Up' or 'Down' prediction for tomorrow.
    probability: float, Probability of the predicted direction.
    """
    # Load the saved model
    model_filename = f"xgboost_{ticker}.joblib"
    model = load(model_filename)
    print(f"Loaded model from {model_filename}")
    
    # Fetch and preprocess recent data
    data = fetch_stock_data(ticker, start_date='2022-01-01')  # Fetch historical data including today
    data = calculate_technical_indicators(data)
    data = data.dropna()  # Remove NaN values
    
    # Use the last row as input for prediction
    latest_data = data.iloc[-1:]  # Get the last row (most recent data)
    
    # Select the features for prediction
    X_latest = latest_data[feature_subset] if feature_subset else latest_data.drop(columns=['Close', 'Volume', 'Open', 'High', 'Low'])
    
    # Predict tomorrow's movement
    y_pred = model.predict(X_latest)[0]
    y_pred_proba = model.predict_proba(X_latest)[0]
    
    # Interpret the prediction
    prediction = 'Up' if y_pred == 1 else 'Down'
    probability = y_pred_proba[1] if y_pred == 1 else y_pred_proba[0]
    
    return prediction, probability

# Example usage
ticker = 'TSLA'

# Use the best features from training
best_features_TSLA = ['WCLPRICE', 'AROON_UP', 'MIDPOINT', 'TYPPRICE', 'MACD', 'BB_upper', 'MACD_hist', 'T3', 'ADX', 'SMA', 'PLUS_DI', 'STOCH_fastk', 'MINUS_DM', 'TEMA', 'ATR', 'STOCH_fastd', 'AROON_DOWN', 'BB_middle', 'NATR', 'HT_LEADSINE', 'MFI', 'OBV', 'HT_PHASOR_inphase', 'STOCH_slowd']  # Replace with your best features

# Predict tomorrow's movement
tomorrow_prediction, tomorrow_probability = predict_tomorrow_xgboost(ticker, feature_subset=best_features_TSLA)

# Print the results
print(f"Prediction for {ticker} tomorrow: {tomorrow_prediction} (Probability: {tomorrow_probability:.2f})")

best_features_MSFT = ['STOCH_fastd', 'PLUS_DM', 'TRANGE', 'HT_TRENDMODE', 'STOCH_fastk', 'ADOSC', 'STOCH_slowd']
ticker = 'MSFT'
# Predict tomorrow's movement
tomorrow_prediction_msft, tomorrow_probability_msft = predict_tomorrow_xgboost(ticker, feature_subset=best_features_MSFT)

# Print the results
print(f"Prediction for {ticker} tomorrow: {tomorrow_prediction_msft} (Probability: {tomorrow_probability_msft:.2f})")


Loaded model from xgboost_TSLA.joblib


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Prediction for TSLA tomorrow: Down (Probability: 0.99)
Loaded model from xgboost_MSFT.joblib
Prediction for MSFT tomorrow: Up (Probability: 0.69)
